In [55]:
import numpy as np
import scipy as sc
from tqdm.notebook import tqdm
from scipy import linalg
from scipy.optimize import minimize
from scipy.sparse import csr_matrix
from scipy.special import expit as sigmoid
from scipy.special import log_expit, softmax, log_softmax
from joblib import Parallel, delayed

In [56]:
log_softmax(np.ones(5))

array([-1.60943791, -1.60943791, -1.60943791, -1.60943791, -1.60943791])

In [57]:
import os
import shutil
import sys

import numpy as np
from scipy import sparse

#import matplotlib.pyplot as plt
#%matplotlib inline
#
#import seaborn as sn
#sn.set()

import pandas as pd

import bottleneck as bn

import datetime
from copy import deepcopy

In [58]:

import pickle

def save_pkl(obj, filename ):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL )
    
def load_pkl(filename ):
    with open(filename, 'rb') as f:
        return pickle.load(f)
 

In [59]:
DATA_DIR = '/efs/users/hsteck/public/data_for_ease/movielens20mio/'

In [60]:
pro_dir = os.path.join(DATA_DIR, 'pro_sg')


In [61]:
unique_sid = list()
with open(os.path.join(pro_dir, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

n_items = len(unique_sid)

def load_train_data(csv_file):
    tp = pd.read_csv(csv_file)
    n_users = tp['uid'].max() + 1

    rows, cols = tp['uid'], tp['sid']
    data = sparse.csr_matrix((np.ones_like(rows),
                             (rows, cols)), dtype='float64',
                             shape=(n_users, n_items))
    return data

In [62]:
def load_xtx_binary(I):
    train_data = load_train_data(os.path.join(pro_dir, 'train.csv'))
    X_reg=train_data[:,:I]
    X = X_reg.toarray()
    Xp = csr_matrix(np.where(X==-1,0,1))
    Xn = csr_matrix(np.where(X==1,0,-1))
    #X = -1.0*csr_matrix.sign(X)
    #X = 2 * X - 1
    ####normalize users
    #nn=np.array(np.sum(X,axis=1)) [:,0]
    #nn=1.0/np.sqrt(nn)  # user weight normalized on diagonal, approx prop to nn  (off diag)
    #X=  sparse.spdiags(nn, 0, len(nn), len(nn)) * X
    ### remove mean  --> cov
    print (X_reg.shape)
    #XtX=np.array(X.T.dot(X).todense()) 

    return [X_reg.shape[0] , X_reg, Xp, Xn]


def load_xtx_sign(I):
    train_data = load_train_data(os.path.join(pro_dir, 'train.csv'))
    
    X=train_data[:,:I]
    X = X.toarray()
    #X = -1.0*csr_matrix.sign(X)
    X = 2 * X - 1
    ####normalize users
    #nn=np.array(np.sum(X,axis=1)) [:,0]
    #nn=1.0/np.sqrt(nn)  # user weight normalized on diagonal, approx prop to nn  (off diag)
    #X=  sparse.spdiags(nn, 0, len(nn), len(nn)) * X
    ### remove mean  --> cov
    print (X.shape)
    #XtX=np.array(X.T.dot(X).todense()) 

    return [X.shape[0] ,X,None]

In [63]:
#I = 20108
I = 200
U = 116677
userCnt , X, Xp, Xn = load_xtx_binary(I)
X_ = X.toarray()
scale = 1
Xt = (2*X_-1)*scale

(116677, 200)


In [64]:
Xt.shape

(116677, 200)

In [65]:
def ease(X, lam):
    print('multiplying matrix')
    G = X.T @ X
    diagIndices = np.diag_indices(G.shape[0])
    G = G + lam * np.eye(G.shape[0])
    print('inverting')
    P = linalg.inv(G)
    print('inverting complete')
    B = P / (-np.diag(P))
    B[diagIndices] = 0.0
    return B

In [66]:


def sllog(w,item,idx0,idx1,lam=200.0):
    return lam * np.inner(w,w) - np.sum(log_expit(X[idx1[0],:]@w)) - np.sum(log_expit(-X[idx0[0],:]@w))


def dsllog(w,item,idx0,idx1,lam=200.0):
    xw = (X@w).reshape(U,1)
    return (sigmoid(xw)-X[:,item]).T@X + 2 * lam * w


In [68]:
def lsq(B, lam = 1.0):
    B = B.reshape(I,I)
    S = X @ B
    loss = sc.linalg.norm(X - S, ord='fro') ** 2 + lam * (sc.linalg.norm(B, ord='fro') ** 2)
    return loss

def dlsq(B, lam = 1.0):
    B = B.reshape(I,I)
    S = X @ B
    return (2 * lam * B - 2 * X.T @ (X - S)).flatten()


# def llog1(w,item,lam=200.0):
#     xw = (X@w)
#     return np.sum(np.logaddexp(-xw/2,xw/2) - X[:,item]/2*(xw)) + lam * np.inner(w,w)


# def dllog1(w,item,lam=200.0):
#     xw = (X@w)
#     scalar = 1/2 * (sigmoid(xw/2) - sigmoid(-xw/2) - X[:,item])
#     return scalar.T@X + 2 * lam * w



# def llog(w,item,lam=200.0):
#     xw = (X@w)
#     return -1.0 * np.sum(log_expit(X[:,item]*xw)) + lam * np.inner(w,w) / 2


# def dllog(w,item,lam=200.0):
#     y = X[:,item]
#     xw = X@w
#     scalar = -1.0 * y * sigmoid(-y*xw)
#     return scalar.T@X + lam*w


# def llog(w,item,idx0,idx1,lam):
#     xw0 = Xp[idx0,:]@w + Xn[idx0,:]@w
#     xw1 = Xp[idx1,:]@w + Xn[idx1,:]@w
#     loss = -1.0 * np.sum((log_expit(xw1))) - np.sum(log_expit(-xw0))
#     return loss + lam * np.inner(w,w) / 2

# def dllog(w,item,idx0,idx1,lam):
#     scalar = sigmoid(Xp@w + Xn@w).reshape(U,1) - X[:,item]
#     return scalar.T@Xp + scalar.T@Xn + lam*w


# def llog(w,item,lam):
#     xw = Xt@w
#     loss = 0.0
#     for u in range(U):
#         xw = np.inner(Xt[u,:],w)
#         for i in range(I):
#             loss += np.logaddexp(0,(1-2*X_[u,i])*xw)
#     return loss + lam/2 * np.inner(w,w)


def lcat(w,item,lam):
    xw = X_@w
    loss = -1.0*np.inner(X_[:,item],log_softmax(xw))
    return loss + lam * np.inner(w,w) / 2
    

def dlcat(w,item,lam):
    xw = Xt@w
    return (softmax(xw) - X_[:,item]).T@X_ + lam * w

def projected_newton(item,lam,alpha = 1/10):
    w = np.zeros(I)
    done = False
    while not done:
        grad = dllog(w,item,lam)
        hess = hllog(w,item,lam)
        update = sc.sparse.linalg.spsolve(hess,grad)
        beta = np.linalg.norm(update)
        w_new = w - 1 / beta * np.log(1 + beta) * update
        #w_new = w_new / (-w_new[item])
        w_new[item] = 0.0
        #w_new=np.clip(w_new,-5,5)
        #print(np.linalg.norm(w_new-w))
        if np.linalg.norm(w_new-w) < 1e-6:
            done = True
            return w_new
        w = w_new
    return w_new

In [69]:
lam = 0.0
def exp(item):
    b = 0.0
    bounds = np.zeros((I,2))
    bounds[:,0] = -100
    bounds[:,1] = 100
    bounds[item,:] = np.array([b,b])
    # bounds = sc.optimize.Bounds(lb=bounds[:,0],ub=bounds[:,1])
    x0 = np.random.uniform(low=-0.0,high=0.0,size=I)
    x0[item] = b
    # clf = LogisticRegression(solver='L-BFGS-B', penalty = 'l2', C=lam)
    # clf.fit(Xt,X_[:,item],bounds=bounds)
    
    sol = minimize(
       fun = lcat, x0=x0, args = (item,lam),method='l-bfgs-b',jac=dlcat, bounds=bounds,
       options={'gtol':1e-12,'ftol':1e-12}
    )
    print(sol, np.linalg.norm(sol.jac))
    return sol.x





In [70]:
W = Parallel(n_jobs=1)(delayed(exp)(item) for item in tqdm(range(I)))
W = np.array(W)

  0%|          | 0/200 [00:00<?, ?it/s]

  message: ABNORMAL_TERMINATION_IN_LNSRCH
  success: False
   status: 2
      fun: 62257.83604243571
        x: [ 0.000e+00  3.020e-02 ...  2.619e-02  2.650e-02]
      nit: 3
      jac: [-5.409e+03 -1.068e+03 ... -3.168e+02 -2.408e+02]
     nfev: 77
     njev: 77
 hess_inv: <200x200 LbfgsInvHessProduct with dtype=float64> 14929.619511249335
  message: ABNORMAL_TERMINATION_IN_LNSRCH
  success: False
   status: 2
      fun: 150929.08779640906
        x: [ 3.256e-02  0.000e+00 ...  3.256e-02  3.256e-02]
      nit: 1
      jac: [-1.068e+03 -1.325e+04 ... -7.116e+02 -9.117e+02]
     nfev: 56
     njev: 56
 hess_inv: <200x200 LbfgsInvHessProduct with dtype=float64> 40942.171843861135
  message: ABNORMAL_TERMINATION_IN_LNSRCH
  success: False
   status: 2
      fun: 125612.2109256484
        x: [ 3.105e-02  3.111e-02 ...  2.432e-02  2.482e-02]
      nit: 8
      jac: [-1.472e+03 -2.551e+03 ... -4.567e+02 -3.818e+02]
     nfev: 134
     njev: 134
 hess_inv: <200x200 LbfgsInvHessProduct with dt

KeyboardInterrupt: 

In [54]:
np.linalg.norm(dlcat(np.zeros(I),1,1.0))

141958.2765303649

In [38]:
k=100
evaluate(W,I,scale,'log',k)

2024-06-24 17:00:02.515985
0 ... 5000
(100,)
5000 ... 10000
(100,)
Test NDCG@100=0.05386 (0.00069)
2024-06-24 17:00:02.610911


[0.0538576744587746, nan, nan]

In [36]:
W_lin = ease(X,lam)
evaluate(W_lin,I,scale,'ease',k)

multiplying matrix
inverting
inverting complete
2024-06-24 16:59:54.739333
0 ... 5000
(100,)
5000 ... 10000
(100,)
Test NDCG@100=0.19303 (0.00178)
2024-06-24 16:59:54.942925


[0.19303411014326652, nan, nan]

In [39]:
W

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [27]:
def load_tr_te_data(csv_file_tr, csv_file_te):
    tp_tr = pd.read_csv(csv_file_tr)
    tp_te = pd.read_csv(csv_file_te)

    start_idx = min(tp_tr['uid'].min(), tp_te['uid'].min())
    end_idx = max(tp_tr['uid'].max(), tp_te['uid'].max())

    rows_tr, cols_tr = tp_tr['uid'] - start_idx, tp_tr['sid']
    rows_te, cols_te = tp_te['uid'] - start_idx, tp_te['sid']

    data_tr = sparse.csr_matrix((np.ones_like(rows_tr),
                             (rows_tr, cols_tr)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    data_te = sparse.csr_matrix((np.ones_like(rows_te),
                             (rows_te, cols_te)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    return data_tr, data_te

In [28]:
test_data_tr, test_data_te = load_tr_te_data(
    os.path.join(pro_dir, 'test_tr.csv'),
    os.path.join(pro_dir, 'test_te.csv'))

In [29]:
N_test = test_data_te.shape[0]
idxlist_test = range(N_test)


In [30]:
def NDCG_binary_at_k_batch(X_pred, heldout_batch, k=20):
    '''
    normalized discounted cumulative gain@k for binary relevance
    ASSUMPTIONS: all the 0's in heldout_data indicate 0 relevance
    '''
    batch_users = X_pred.shape[0]
    idx_topk_part = bn.argpartition(-X_pred, k, axis=1)
    topk_part = X_pred[np.arange(batch_users)[:, np.newaxis],
                       idx_topk_part[:, :k]]
    idx_part = np.argsort(-topk_part, axis=1)
    # X_pred[np.arange(batch_users)[:, np.newaxis], idx_topk] is the sorted
    # topk predicted score
    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]
    # build the discount template
    tp = 1. / (np.log2(np.arange(2, k + 2)))
    print(tp.shape)

    DCG = (heldout_batch[np.arange(batch_users)[:, np.newaxis],
                         idx_topk].toarray() * tp).sum(axis=1)
    IDCG = np.array([(tp[:min(n, k)]).sum()
                     for n in heldout_batch.getnnz(axis=1)])
    return DCG / IDCG

In [31]:
def Recall_at_k_batch(X_pred, heldout_batch, k=100):
    batch_users = X_pred.shape[0]

    idx = bn.argpartition(-X_pred, k, axis=1)
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True
    
    X_true_binary = (heldout_batch > 0).toarray()
    X_true_binary = X_true_binary[:,:I]
    print(X_true_binary.sum(axis=1))
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(
        np.float32)
    recall = tmp / np.minimum(k, X_true_binary.sum(axis=1))
    return recall

In [33]:
def evaluate(BB,I,scale,method='log',k=100):
    #evaluate in batches
    print(datetime.datetime.now())

    #makeSparseFormat(BB, 0.0)


    batch_size_test=5000
    n100_list, r20_list, r50_list = [], [], []



    for bnum, st_idx in enumerate(range(0, N_test, batch_size_test)):
        end_idx = min(st_idx + batch_size_test, N_test)
        Xtest = test_data_tr[idxlist_test[st_idx:end_idx]]
        Xtest=Xtest[:,:I].toarray()
        print (str(st_idx)+' ... '+str(end_idx))
        if sparse.isspmatrix(Xtest):
            Xtest = Xtest.toarray()
        Xtest = Xtest.astype('float32')

        #pred_val = Xtest.dot(BB_excl)
        #pred_val = (((Xtest-mu) * scaling).dot(BBth) / scaling) +mu   # no bias
        #pred_val = Xtest.dot(beta_0d)  # no bias
        #pred_val =Xtest.dot(beta_lowrank)
        if method == 'log':
            pred_val = softmax((2*Xtest-1).dot(BB))
        else:
            pred_val = Xtest.dot(BB)

        # exclude examples from training and validation (if any)
        pred_val[Xtest.nonzero()] = -np.inf
        n100_list.append(NDCG_binary_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k))
        #r20_list.append(Recall_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=20))
        #r50_list.append(Recall_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=50))
        #calc_coverageCounts(coverageCounts2, pred_val)
        #break  # do only 5000 users

    n100_list = np.concatenate(n100_list)
    #r20_list = np.concatenate(r20_list)
    #r50_list = np.concatenate(r50_list)

    print("Test NDCG@100=%.5f (%.5f)" % (np.mean(n100_list), np.std(n100_list) / np.sqrt(len(n100_list))))
    #print("Test Recall@20=%.5f (%.5f)" % (np.mean(r20_list), np.std(r20_list) / np.sqrt(len(r20_list))))
    #print("Test Recall@50=%.5f (%.5f)" % (np.mean(r50_list), np.std(r50_list) / np.sqrt(len(r50_list))))

    print(datetime.datetime.now())
    return [np.mean(n100_list), np.mean(r20_list), np.mean(r50_list)]

In [31]:
evaluate(W)

TypeError: evaluate() missing 1 required positional argument: 'I'

In [31]:
evaluate(W.T)

2024-06-21 21:56:25.686416
0 ... 5000
5000 ... 10000
Test NDCG@100=0.35762 (0.00214)
2024-06-21 21:56:26.061809


[0.35762368625418844, nan, nan]

In [40]:
evaluate(ease(X_reg,lam=200.0))

multiplying matrix
inverting
inverting complete
2024-06-21 21:58:13.024974
0 ... 5000
5000 ... 10000
Test NDCG@100=0.36168 (0.00215)
2024-06-21 21:58:13.504854


[0.36167687300486134, nan, nan]

In [ ]:
# #Bn_ = (np.ones((I , I)) - 3).flatten()
# Bn = (np.zeros((I , I))).flatten()
# Bn1 = (np.ones((I , I)) + 2).flatten()
# for i in range(10000):
#     #Bnvec = Bn.flatten()
#     #Bn_vec = Bn_.flatten()
#     dBn_ = der(Bn_).flatten()
#     dBn = der(Bn).flatten()
#     top = np.abs(np.inner(Bn - Bn_, dBn - dBn_))
#     bottom = np.linalg.norm(dBn - dBn_) ** 2
#     if bottom <= 1e-132:
#         break
#     gamma = top / bottom
#     #print(gamma)
#     print(np.linalg.norm(der(Bn)))
#     Bn1 = Bn - gamma * der(Bn)
#     Bn_ = Bn
#     Bn = Bn1

# def temp(W):
#     W = W.reshape(I,I)
#     sums = 0.0
#     for i in range(I):
#         xw = X@W[:,i]
#         sums += np.sum( np.logaddexp(-xw/2,xw/2) - X[:,i]/2 * xw)
#     return sums



# def temp2(W, lam = 1.0):
#     W = W.reshape(I,I)
#     S = X @ W
#     return np.sum(np.logaddexp(-S/2,S/2) - X / 2 * S) + lam * np.linalg.norm(W,ord='fro') ** 2


# def der2(W, lam = 1.0):
#     W = W.reshape(I,I)
#     S = X @ W
#     return (1/2 * X.T @ (sigmoid(S/2) - (sigmoid(-S/2)) + X) + 2 * lam * W).flatten()
    



# def der(W):
#     W = W.reshape(I,I)
#     der = np.zeros((I,I))
#     for u in range(U):
#         for i in range(I):
#             scalar = 1 / 2 * (np.tanh(np.inner(X[u],W[:,i])) - X[u,i])
#             der[i] = der[i] + scalar * X[u]
#     return der.flatten()

# def Hess(W):
#     hess = np.zeros((I,I))
#     for u in range(U):
#         for i in range(I):
#             scalar = 1 / (2 * np.cosh(np.inner(X[u],W[:,i])) + 2)
#             hess = hess + scalar * np.outer(X[u],X[u])
#     return hess

# def lsq(B, X, lam = 1.0):
#     B = B.reshape(I,I)
#     S = X @ B
#     loss = sc.linalg.norm(X - S, ord='fro') ** 2 + lam * (sc.linalg.norm(B, ord='fro') ** 2)
#     return loss

# def dlsq(B, X, lam = 1.0):
#     B = B.reshape(I,I)
#     S = X @ B
#     return (2 * lam * B - 2 * X.T @ (X - S)).flatten()


# def llog(Bvec, X, lam = 1.0):
#     B = Bvec.reshape(I,I)
#     S = X @ B
#     Svec = S.flatten()
#     Xvec = X.flatten()

#     loss = np.sum(np.logaddexp(-Svec/2,Svec/2) - Xvec/2*Svec)
    
#     loss = lam * np.inner(Bvec,Bvec) + loss
#     return loss


# def llog2(Bvec, X, lam = 1.0):
#     B = Bvec.reshape(I,I)
#     S = X @ B
#     Svec = S.flatten()
#     Xvec = X.flatten()

#     loss = np.sum(np.logaddexp(-S/2,S/2) - X/2*S)
    
#     loss = lam * np.inner(Bvec,Bvec) + loss
#     return loss

# def dllog2(Bvec, X, lam = 1.0):
#     B = Bvec.reshape(I,I)
#     S = X @ B
#     return (1/2 * X.T @ (2*sigmoid(S) - np.ones((U,I)) + X) + 2 * lam * B).flatten
    
    
     

In [ ]:
# Bn = np.zeros(I * I)  
# Bn_ = np.zeros(I * I) + 10 
# Bn1 = np.zeros(I * I) + 100
# gamma = 1.0
# lam = 0.0
# while gamma > 1e-10:
#     dBn_ = der2(Bn_)
#     dBn = der2(Bn)
#     top = np.abs(np.inner(Bn - Bn_, dBn - dBn_))
#     bottom = np.linalg.norm(dBn - dBn_) ** 2
#     if bottom <= 1e-32:
#         break
#     gamma = top / bottom
#     print(temp2(Bn1),np.linalg.norm(der2(Bn1)),gamma)
#     Bn1 = Bn - gamma * der2(Bn)
#     #Bn1mat = Bn1.reshape(I,I)
#     #Bn1 = (Bn1mat - np.diag(np.diag(Bn1mat))).flatten()
#     Bn_ = Bn
#     Bn = Bn1

In [ ]:
# print(temp2(Bn1),np.linalg.norm(der2(np.eye(I) * 100)))

In [ ]:
# W = ease(X)

In [ ]:
# np.linalg.norm(W - Bn1.reshape(I,I))

In [4]:
U = 200_000
I = 90
lam = 100
#X = np.zeros((U,I))

p = np.random.uniform(size=I)
print('sampling matrix')
X = csr_matrix(np.random.binomial(n=1,p=p,size=(U,I)))

sampling matrix


In [50]:
np.where(X[:,0]==0)

/apps/bdi-venv-310/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3553: SparseEfficiencyWarning: Comparing a sparse matrix with 0 using == is inefficient, try using != instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all().

In [51]:
X[:,0]

<116677x1 sparse matrix of type '<class 'numpy.float64'>'
	with 5409 stored elements in Compressed Sparse Row format>

In [52]:
from sklearn.metrics import log_loss

In [53]:
log_loss(X[:,0],sigmoid(X@np.zeros(I)))

TypeError: unhashable type: 'csr_matrix'

In [227]:
X = 2 * np.random.binomial(1,0.5,size=(5,5)) - 1

In [239]:
Xp = np.where(X==-1,0,1)
Xn = np.where(X==1,0,-1)

w = np.random.uniform(size=5)


In [240]:
X@w

array([ 1.20294548,  1.51995758, -1.60616126,  1.60616126, -2.16785936])

In [241]:
Xp@w + Xn@w

array([ 1.20294548,  1.51995758, -1.60616126,  1.60616126, -2.16785936])